In [1]:
# 1. 检索相似问题
# 一个相似问题对应5个id

# 2. 根据相似问题path标签获取文本块
# 返回文本块


In [2]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(
    api_key="sk-EQY6oy2D9Brb2IWxymHTnmeFz5unIyobzHtjDgFs2ZwEPjmY",
    base_url="https://api.chatanywhere.tech/v1",
    model="gpt-4o-mini",
    temperature=0.35,
)
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate
)

In [3]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model_path = r"D:/Python/models/bge-large-zh-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_path)

C:\Users\pixiu\AppData\Local\Temp\ipykernel_14088\534697663.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model_path)
d:\Python\MiniConda\envs\itext2kg\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_community.vectorstores import FAISS
import pickle

def loader_faiss(faiss_index_path, embeddings_model_path):
    """
    加载 FAISS 索引
    :param faiss_index_path: FAISS 索引路径
    :param embeddings_model_path: 嵌入模型路径
    :return: FAISS 向量数据库对象
    """
    # 加载 FAISS 索引
    index = faiss.read_index(faiss_index_path + 'faiss_index.index')
    with open(faiss_index_path + "docstore.pkl", "rb") as f:
        docstore = pickle.load(f)

    with open(faiss_index_path + "index_to_docstore_id.pkl", "rb") as f:
        index_to_docstore_id = pickle.load(f)

    # 从加载的索引创建 FAISS 向量数据库
    vector_store_loaded = FAISS(
        embedding_function=HuggingFaceEmbeddings(model_name=embeddings_model_path),
        index=index,
        docstore=docstore,
        index_to_docstore_id=index_to_docstore_id
    )
    return vector_store_loaded

In [5]:
embeddings_model_path = r"D:/Python/models/bge-large-zh-v1.5"
q_index_path = 'D:\\code\\agent_demo\\kg\\make_index\\questions\\'
c_index_path = 'D:\\code\\agent_demo\\kg\\make_index\\book\\'
q_vector_store_loaded = loader_faiss(q_index_path, embeddings_model_path)
c_vector_store_loaded = loader_faiss(c_index_path, embeddings_model_path)

query = "学科知识是教师所应具备的专业知识的主体。"
search_results = q_vector_store_loaded.similarity_search(query, k=5)  # k 是返回的结果数量

for result in search_results:
    print(result)

page_content='在教师专业知识结构中，[[本体性知识/学科知识]]是指教师具有的特定的学科知识，是教师知识结构的核心。nan' metadata={'difficult_value_id': 2.0, 'knowledge_point_ids': '367611176748560399_wx', 'path': '[[2005, 3015, 188, 190, 196], [2005, 3015, 188, 190, 196, 198], [2005, 3015, 212, 213, 215, 216], [2005, 3015, 188, 190, 196, 200], [2005, 3015, 188, 190, 196, 197]]', 'answer': '["本体性知识\\/学科知识"]', 'analysis': nan}
page_content='[[学科教学知识]]又称为教学内容知识，这是教师专业知识的核心、重要而最有用的成分。nan' metadata={'difficult_value_id': 2.0, 'knowledge_point_ids': '367611176748560399_wx', 'path': '[[2005, 3015, 188, 190, 196, 199], [2005, 3015, 188, 190, 196, 198], [2005, 3015, 188, 190, 196, 197], [2005, 3015, 188, 190, 196], [2005, 3015, 212, 213, 215, 216]]', 'answer': '["学科教学知识"]', 'analysis': nan}
page_content='教师专业知识主要包括本体性知识、[[条件性知识]]、实践性知识和一般文化知识。nan' metadata={'difficult_value_id': 2.0, 'knowledge_point_ids': '367611176748560399_wx', 'path': '[[2005, 3015, 188, 190, 196], [2005, 3015, 212, 213, 215, 216], [2005, 3015, 188, 190, 196, 198], [2005, 

In [6]:
def get_concept_by_question(question, embeddings_model_path, q_index_path, c_index_path, k=3):
    """
    根据问题获取概念
    :param question: 问题
    :param vector_store_loaded: 向量数据库对象
    :param k: 返回的结果数量
    :return: 概念列表
    """
    q_vector_store_loaded = loader_faiss(q_index_path, embeddings_model_path)
    c_vector_store_loaded = loader_faiss(c_index_path, embeddings_model_path)
    
    import json
    with open("neo4j_export.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    
    node_dict = {node["id"]: node for node in data["nodes"]}

    # 进行相似性搜索
    sim_q = q_vector_store_loaded.similarity_search(question, k=k)
    # sim_c = c_vector_store_loaded.similarity_search(question, k=k)

    source_list = []
    for q in sim_q:
        import ast
        meatdata = q.metadata
        path = ast.literal_eval(meatdata["path"])
        for p in path:
            source_list.append(p)

    result_list = []
    from collections import Counter
    
    source_tuples = [tuple(x) for x in source_list]
    # 用 Counter 统计频率
    counter = Counter(source_tuples)
    
    sorted_unique_sources = [list(item) for item, count in counter.most_common()]
    for path in sorted_unique_sources:
        source_id = path[-1]
        print(path)
        c_node = node_dict[source_id]
        
        node1 = node_dict[path[0]]
        node2 = node_dict[path[1]]
        path_str = '第' + str(node1['properties']['num']) + '章' + node1['properties']['title'] + '\n'
        path_str += '第' + str(node2['properties']['num']) + '节' + node2['properties']['title'] + '\n'
        for p in path[2:]:
            node = node_dict[p]
            path_str += '----' + node['properties']['name'] + '----\n'
        
        result_list.append((c_node['properties']['name'] + c_node['properties']['page_content'], path_str))

    return result_list


In [7]:
query = "学科知识是教师所应具备的专业知识的主体。"
result = get_concept_by_question(query, embeddings_model_path, q_index_path, c_index_path, k=3)
print(result)

[2005, 3015, 188, 190, 196]
[2005, 3015, 188, 190, 196, 198]
[2005, 3015, 212, 213, 215, 216]
[2005, 3015, 188, 190, 196, 200]
[2005, 3015, 188, 190, 196, 197]
[2005, 3015, 188, 190, 196, 199]
[2005, 3015, 212, 220, 221]
[('专业知识\n教师的专业知识是教师职业区别于其他职业的理论体系与经验系统。教师专业知识的质的深化，体现了教师职业的专业性，它是衡量教师专业化程度的标志之一。教师应具备以下几方面的知识。', '第6章教师\n第2节教师的专业素养与专业发展\n----教师的专业素养----\n----教师专业素养的结构----\n----专业知识----\n'), ('学科知识\n学科知识是教师所应具备的专业知识的主体。教师要想顺利完成某一学科复杂的教学活动，必须对该学科的内容有深入透彻的了解，精通该学科的知识。教师不仅要理解和掌握所教学科的基本事实、概念、原理、理论和学科框架以及探究或思维的方式，还要了解和熟练掌握所教学科的最新发展以及与其他学科和社会实践的联系。', '第6章教师\n第2节教师的专业素养与专业发展\n----教师的专业素养----\n----教师专业素养的结构----\n----专业知识----\n----学科知识----\n'), ('教育知识技能的体系化。即形成一套关于教育教学的专业知识技能体系，作为教师教育的内容和教师从事教育教学工作的依据。教师职业专业化的知识技能体系既包括学科专业知识技能，也包括教育专业知识技能。', '第6章教师\n第2节教师的专业素养与专业发展\n----教师专业发展----\n----教师专业发展概述----\n----教师群体专业化----\n----教育知识技能的体系化----\n'), ('通识性知识\n通识性知识是教师应具备的基础知识，包括自然学科、人文社会学科、人文艺术学科以及适应教育内容、教学手段和方法现代化的信息技术知识。广博的文化修养具有陶冶人文精神、养成人文素养的内在价值。教师拥有广博的一般文化科学知识，能够适应学生的好奇心和求知欲，拓展学生的精神世界，发展学生智慧，从而使学生内

In [33]:
def match_best_son(father, sons, question):
    """
    匹配最好的儿子
    :father: 父亲节点
    :param sons: 儿子列表
    :param question: 问题
    :return: 最好的儿子
    """
    
    prompt = '''
-任务-
你是一个自然语言处理专家。你将会被给到一个问题、一个父节点和多个子节点的信息，你的任务是根据父节点和子节点的信息，分析出最符合问题的节点，并给出分析过程。

-输出-
输出内容为json 格式，包含以下字段：
- best_son: 最符合问题的节点id（从子节点id和父节点id中选择）
- analysis: 你是如何选择这个节点的？请详细描述你的分析过程。

'''

    prompt = prompt.replace('{', '{{').replace('}', '}}')
    system_message_prompt = SystemMessagePromptTemplate.from_template(prompt)

    human_template="""
    -真实数据- 
    ######################
    问题：{question}
    父节点：{father}
    子节点：{input_text}
    ###################### 
    输出：
    """

    from langchain.chains import LLMChain
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_template])

    chain = LLMChain(llm=llm, prompt=chat_prompt)

    text = ""
    father_text = father['name'] + " " + father['properties']['description'] + " " + father['properties']['page_content']
    for node in sons:
        text += "id:" + str(node['id']) + "、名称：" + node['name'] + " 描述：" + node['properties']['description'] + " 内容：" + node['properties']['page_content'] + "\n"
    
    answer = chain.invoke({
        "question": question,
        "father": father_text,
        "input_text": text
        })
    # print(answer['text'])
    
    import re
    import json
    text = re.sub(r"^```json\s*|\s*```$", "", answer['text'].strip(), flags=re.IGNORECASE)
    json_data = json.loads(text)
    
    return json_data

In [45]:
def get_concept_by_question_v2(question, embeddings_model_path, q_index_path, c_index_path, k=5):
    """
    根据问题获取概念
    :param question: 问题
    :param vector_store_loaded: 向量数据库对象
    :param k: 返回的结果数量
    :return: 概念列表
    """
    q_vector_store_loaded = loader_faiss(q_index_path, embeddings_model_path)
    c_vector_store_loaded = loader_faiss(c_index_path, embeddings_model_path)
    
    def find_lca(paths):
    # Transpose the list of paths, zip stops at shortest path
        zipped = list(zip(*paths))
        lca = []
        for level in zipped:
            if all(x == level[0] for x in level):
                lca.append(level[0])
            else:
                break
        return lca

    # sim_q = q_vector_store_loaded.similarity_search(question, k=k)
    sim_c = c_vector_store_loaded.similarity_search(question, k=k)

    result_list = []
    path_list = []
    for q in sim_c:
        # print(q)
        path = q.metadata["path"]
        path_list.append(path)
    
    lac = find_lca(path_list)
    # print("Lowest Common Ancestor:", lac)
    
    from collections import defaultdict
    import json
    with open("neo4j_export.json", "r", encoding="utf-8") as f:
        data = json.load(f)
    
    node_dict = {node["id"]: node for node in data["nodes"]}
    reverse_edges = defaultdict(list)
    for edge in data["edges"]:
        reverse_edges[edge["target"]].append(edge["source"])
        parent_to_children = defaultdict(list)
        child_to_parent = {}

    for edge in data["edges"]:
        if edge["type"] == "包含":
            parent, child = edge["source"], edge["target"]
            parent_to_children[parent].append(child)
            child_to_parent[child] = parent

    last_node_id = lac[-1]
    while True:
        sons = parent_to_children.get(last_node_id, [])
        # print("sons:", sons)
        if sons:
            son_list = []
            for son in sons:
                son_list.append(node_dict[son])
            
            # print("son_list:", son_list)
            last_node = match_best_son(node_dict[last_node_id], son_list, question)
        else:
            break
        
        # print (last_node)
        if last_node['best_son'] == last_node_id:
            break
        
        last_node_id = int(last_node['best_son'])
    
    last_node = node_dict[last_node_id]
    last_sons = parent_to_children.get(last_node_id, [])
    # 递归函数：向上查找路径
    def trace_to_root(node_id, visited=None):
        if visited is None:
            visited = set()
        if node_id in visited:
            return []  # 避免环
        visited.add(node_id)
        
        if node_id not in reverse_edges:
            return [node_id]  # 到达顶点
        # 默认选择第一条路径（如果多个前驱）
        parent = reverse_edges[node_id][0]
        return trace_to_root(parent, visited) + [node_id]
    
    def get_path(node_id, node_dict):
        # print(node_dict[node_id])
        # 构建节点信息和反向边字典（target -> source）
        node_dict = {node["id"]: node for node in data["nodes"]}
        reverse_edges = defaultdict(list)
        for edge in data["edges"]:
            reverse_edges[edge["target"]].append(edge["source"])

        path = trace_to_root(node_id)
        
        path = path[1:]  # 反转路径
        node1 = node_dict[path[0]]
        node2 = node_dict[path[1]]
        
        path_str = '第' + str(node1['properties']['num']) + '章' + node1['properties']['title'] + '\n'
        path_str += '第' + str(node2['properties']['num']) + '节' + node2['properties']['title'] + '\n'
        for p in path[2:]:
            node = node_dict[p]
            path_str += '----' + node['properties']['name'] + '----\n'
        return (last_node['properties']['page_content'], path_str)

    res = []
    res.append(get_path(last_node_id, node_dict))

    if last_sons:
        for son in last_sons:
            res.append(get_path(son, node_dict))

    return res

get_concept_by_question_v2("提出“知之者不如好之者”命题的思想家是（）", embeddings_model_path, q_index_path, c_index_path, k=5)

[('\n孔子的教育思想主要体现在《论语》中，其思想主要有以下几个方面。\n①论教育作用\n论教育的社会功能：“庶”“富”“教”是孔子的治国大纲，他认为，人口、经济与教育是立国、治国的三大要素，教育对社会发展有重要作用。孔子是中国教育史上最先论述教育与经济发展关系的教育家。\n论教育的个体功能：孔子从探讨人的本性入手，认为人的先天本性相差不大，个性的差异主要是由后天形成的。如：“性相近也，习相远也。”\n②论教育对象 “有教无类”\n孔子创办私学，主张“有教无类”，即不分贵族与平民，都可以接受教育，成为教育的对象，打破了贵族垄断教育的局面。\n③论培养目标——“学而优则仕”\n孔子的教育目的就是要培养德才兼备的君子。“学而优则仕”口号的提出，确定了培养统治人才这一教育目的，在教育史上有重要的意义。\n④论教育内容——“仁”“礼”为核心\n孔子以“礼”为道德规范，以“仁”为最高道德标准，主张立志、克己、内省、力行、改过，开创了以儒学伦理为核心，以道德教育为本的教育传统。孔子整理“六经”，即《诗》《书》《礼》《乐》《易》《春秋》六种教材，奠定了儒家教育内容的基础。在教育内容上，孔子主张“子以四教:文、行、忠、信”。孔子的教学内容偏重社会人事、偏重文事，轻视科技与生产劳动。\n⑤教学纲领——“博学于文，约之以礼”\n⑥论教育方法\n第一，启发诱导。孔子曾说过：“不愤不启，不不发。举一隅不以三隅反，则不复也。”孔子是世界上最早提出启发式教学的教育家。\n第二，因材施教。孔子因材施教的前提是承认学生间的个体差异，并了解学生特点，在了解的基础上有针对性地进行教育。如“求也退，故进之；由也兼人，故退之。”\n⑦论学习方法\n第一，温故知新。“学而时习之”“温故而知新，可以为师矣。”\n第二，学思结合、学行结合。即学思并重，“学而不思则罔，思而不学则殆”，孔子强调学习知识要“学以致用”。学、思、行相结合也是孔子所探究和总结的学习过程，对后来的教学理论、教学实践产生了深远影响。',
  '第1章教育与教育学\n第2节教育学概述\n----教育学的产生与发展----\n----教育学的萌芽阶段----\n----中国古代教育思想----\n----孔子----\n')]